In [ ]:
# Check GPU Specs
!nvidia-smi

In [11]:
# Get hashes to crack
# Make a directory called 'dothashcat' in your Google Drive before this
# with the 'hashes' subdirectory for storing hashes
from google.colab import drive

# This will ask you to go to a link and get an authorization code
# to let the .hashcat folder get synced to your Google Drive
drive.mount('/content/drive')

# Make a symbolic link between Google Drive
# and the local .hashcat directory
!ls /content/drive/My\ Drive/dothashcat
!ln -s /content/drive/My\ Drive/dothashcat /root/.hashcat

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ls: cannot access '/content/drive/My Drive/dothashcat': No such file or directory


In [12]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.25.1-1ubuntu3.10).
Suggested packages:
  gettext
The following NEW packages will be installed:
  checkinstall
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 99.3 kB of archives.
After this operation, 442 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 checkinstall amd64 1.6.2+git20170426.d24a630-2ubuntu1 [99.3 kB]
Fetched 99.3 kB in 1s (128 kB/s)
Selecting previously unselected package checkinstall.
(Reading database ... 122352 files and directories currently installed.)
Preparing to unpack 

In [13]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat --version

v6.2.6-463-g41a4d8c9c


In [ ]:
# WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat -w 4 -m 2500 --benchmark

In [ ]:
# Get wordlists
!mkdir /root/wordlists && cd /root/wordlists

# Clone the SecLists repository
!git clone https://github.com/danielmiessler/SecLists /root/wordlists/SecLists

# Append 'hashcat' for demonstration purposes
!echo "hashcat!" >> /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

!ls /root/wordlists/SecLists

In [ ]:
# Check that your hashes are accessible to Colab
!ls /root/.hashcat/hashes

In [ ]:
# Start pure brute force cracking with a session name
!hashcat -m 22000 -a3 --session 1 -status -o wifound.txt /content/cap_files/witest.22000
# !hashcat -m 2500 /root/.hashcat/hashes/test.hccapx -a 3 -w 4 --session test-session

# Restore the session
# !hashcat --session $SESSION --restore

# Crack with a wordlist
#!hashcat -m 2500 /root/.hashcat/hashes/test.hccapx /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

hashcat (v6.2.6-463-g41a4d8c9c) starting

clGetPlatformIDs(): CL_PLATFORM_NOT_FOUND_KHR

nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 12.0)
* Device #1: Tesla T4, 14998/15101 MB, 40MCU

Minimum password length supported by kernel: 8
Maximum password length supported by kernel: 63

Hashes: 2 digests; 2 unique digests, 1 unique salts
Bitmaps: 16 bits, 65536 entries, 0x0000ffff mask, 262144 bytes, 5/13 rotates

Optimizers applied:
* Zero-Byte
* Single-Salt
* Brute-Force
* Slow-Hash-SIMD-LOOP

Watchdog: Temperature abort trigger set to 90c

Host memory required for this attack: 1470 MB

Finished self-test

In [10]:
# creating new directory to store cap files
!mkdir cap_files
# uploading cap
from google.colab import files

uploaded = files.upload()

for f_name in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=f_name, length=len(uploaded[f_name])))

!mv {f_name} "/content/cap_files/{f_name}"

KeyboardInterrupt: ignored

In [ ]:
import glob
import subprocess
import re
from tqdm import tqdm
is_cracked = False
wordlists = list()
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Common-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Cracked-Hashes/milw0rm-dictionary.txt'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Default-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Honeypot-Captures/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Leaked-Databases/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Malware/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/WiFi-WPA/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Software/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/*.txt'):
  wordlists.append(wordlist)

In [ ]:
def check_if_cracked(command): 
  output = subprocess.check_output([command + " --show"], shell=True).decode()
  if len(output) == 0:
    return False
  else:
    return True



In [ ]:
cap_file = f"/content/cap_files/{f_name} "
for wordlist in tqdm(wordlists):
  command = 'hashcat -m 2500 ' + cap_file + wordlist
  subprocess.call([command],shell=True)
  if check_if_cracked(command):
    is_cracked = True
  if is_cracked:
    print("Password is cracked!!")
    cracked_password = subprocess.check_output([command + " --show"], shell=True).decode()
    cracked_password = cracked_password.split(':')[-1]
    print(f"Password : {cracked_password}")
    print(f"wordlist used : {wordlist}")
    break
  else:
    continue
  
if is_cracked == False:
  print("Password was not found in SecLists")
else:
  pass